In [52]:
import numpy as np
import dask
import dask.array as da
%load_ext line_profiler

In [67]:
def dcg(A, b, tol=1e-8, maxiter=500, x0=None, preconditioner=None, verbose=0):
    """ Conjugate gradient
        
        Parameters
        ----------
        A: array-like
        b: array-like
        tol: float
        
        Returns
        -------
        x: array-like
        iters: int
        resnorm: float
        
        Find x such that

            Ax = b
        
        for square, symmetric A.

        If a preconditioner M is provided, solve the left-preconditioned
        equivalent problem,

            M(Ax - b) = 0
    """
    print_iter = max(1, maxiter / 10**verbose)
        
    A, b, M = dask.persist(A, b, preconditioner)

    if x0 is None:
        r = 1 * b
        x = 0 * b
    else:
        r = 1 * b - A.dot(x0)
        x = x0
        
    Mr = r if M is None else M.dot(r)
    p = Mr
    resnrm2 = r.dot(Mr)
            
    x, r, p, resnrm2 = dask.persist(x, r, p, resnrm2)
    (resnrm2,) = dask.compute(resnrm2)
    if resnrm2**0.5 < tol:
        return x, 0, resnrm2**0.5
    
    for k in range(maxiter):
        ox, ores, op, oresnrm2 = x, r, p, resnrm2

        Ap = A.dot(p)
        alpha = resnrm2 / p.dot(Ap)
        x = ox + alpha * p
        r = ores - alpha * Ap
        Mr = r if M is None else M.dot(r)
        resnrm2 = r.dot(Mr)
        
        x, r, resnrm2 = dask.persist(x, r, resnrm2)
        (resnrm2,) = dask.compute(resnrm2)
        
        if resnrm2**0.5 < tol:
            break
        elif (k + 1) % print_iter == 0:
            print("ITER: {:5}\t||Ax -  b||_2: {}".format(k + 1, resnrm2**0.5))

        p = Mr + (resnrm2 / oresnrm2) * op
        x, r, resnrm2, p= dask.persist(x, r, resnrm2, p)

        (p,) = dask.persist(p)
            
    return x, k + 1, resnrm2**0.5 

In [18]:
m = 400
n = 300
mc = 100
nc = 100

In [19]:
A = np.random.random((m, n))
x = np.random.random(n)
b = np.random.random(n)
rho = 1.

In [20]:
Asymm_unregularized = A.T.dot(A)

In [26]:
xcg, iters, res = dcg(Asymm_unregularized, b, verbose=2)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITER:     5	||Ax -  b||_2: 2.3489558193104463
ITER:    10	||Ax -  b||_2: 1.341780725236021
ITER:    15	||Ax -  b||_2: 0.6957766291123976
ITER:    20	||Ax -  b||_2: 0.40292975749993964
ITER:    25	||Ax -  b||_2: 1.0675210587298722
ITER:    30	||Ax -  b||_2: 0.15369097940349047
ITER:    35	||Ax -  b||_2: 0.06487400113382467
ITER:    40	||Ax -  b||_2: 0.03407526384964162
ITER:    45	||Ax -  b||_2: 0.01621768783646738
ITER:    50	||Ax -  b||_2: 0.008922389927080906
ITER:    55	||Ax -  b||_2: 0.004461073554402745
ITER:    60	||Ax -  b||_2: 0.0024406219730122684
ITER:    65	||Ax -  b||_2: 0.017398720176550635
ITER:    70	||Ax -  b||_2: 0.0006825204855129539
ITER:    75	||Ax -  b||_2: 0.00026378477422641587
ITER:    80	||Ax -  b||_2: 0.0001319352201061976
ITER:    85	||Ax -  b||_2: 7.08414252391683e-05
ITER:    90	||Ax -  b||_2: 3.081490595557912e-05
ITER:    95	||Ax -  b||_2: 1.4089639810193372e-05
ITER:   100	||Ax -  b||_2: 6.331693747459421e-06
ITER:   105	||Ax -  b||_2: 3.5050873136593183

In [27]:
Asymm_weakreg = A.T.dot(A) + 0.1 * np.eye(n)

In [28]:
xcg, iters, res = dcg(Asymm_weakreg, b)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:  137
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.363916371989383e-10


In [29]:
Asymm = A.T.dot(A) + np.eye(n)

In [30]:
xcg, iters, res = dcg(Asymm, b, verbose=1)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITER:    50	||Ax -  b||_2: 0.0002859816929092584
ITERS:   95
||(rho * I + A'A)x - b||_2 / sqrt(n): 4.647475130418556e-10


In [31]:
xcg, iters, res = dcg(Asymm, b, preconditioner=np.eye(n))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   95
||(rho * I + A'A)x - b||_2 / sqrt(n): 4.647475130418556e-10


In [33]:
class JacobiPrecond(object):
    """ Build a Jacobi diagonal preconditioner P = inv(diag(A))
    """
    def __init__(self, A_symmetric):
        self.d = np.reciprocal(np.diag(A_symmetric))
    def dot(self, v):
        return self.d * v

In [66]:
xcg, iters, res = dcg(Asymm, b, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

TYPE BEFORE PERSIST <class 'numpy.float64'>
TYPE AFTER PERSIST <class 'numpy.float64'>
ITERS:   84
||(rho * I + A'A)x - b||_2 / sqrt(n): 4.767819726779389e-10


In [35]:
## WARMSTART

In [36]:
pct_perturb = 5.
perturb = np.random.random(n) - 0.5
perturb = (perturb / np.linalg.norm(perturb)) * (pct_perturb / 100.) * np.linalg.norm(b) 
b_new = b + perturb

In [37]:
xwarm, iters, res = dcg(Asymm, b_new, x0=xcg, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   75
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.739212271192919e-10


In [38]:
xcold, iters, res = dcg(Asymm, b_new, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   85
||(rho * I + A'A)x - b||_2 / sqrt(n): 4.688996932499014e-10


In [39]:
xwarm_noprecon, iters, res = dcg(Asymm, b_new, x0=xcg)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   87
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.347839043461314e-10


In [40]:
xcold_noprecon, iters, res = dcg(Asymm, b_new)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   96
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.382232368346334e-10


In [49]:
Ad = da.from_array(Asymm, chunks=150)

In [61]:
%lprun -f dcg dcg(Ad, b)

In [64]:
%lprun -f dcg dcg(Asymm, b)

In [ ]:
mlg = 40000
nlg = 30000
Alg = np.random.random((mlg, nlg))
Aslg = Alg.T.dot(Alg) + np.eye(nlg)
blg = np.random.random(nlg)
Adlg = da.from_array(Aslg, chunks=nlg)